<a href="https://colab.research.google.com/github/aelitta/parse_rsbu/blob/main/parse_pdf_rsbu_colab_nozip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install libmagickwand-dev
!apt install imagemagick
!sudo apt install tesseract-ocr
!pip install pytesseract
!sed -i 's/<policy domain="coder" rights="none" pattern="PDF" \/>/<policy domain="coder" rights="read|write" pattern="PDF" \/>/g' /etc/ImageMagick-6/policy.xml
!sed -i 's/<policy domain="resource" name="disk" value="16GB" \/>/<policy domain="resource" name="disk" value="32GB" \/>/g' /etc/ImageMagick-6/policy.xml
!sed -i 's/<policy domain="resource" name="memory" value="256MiB" \//><policy domain="resource" name="memory" value="1024MiB" \/>/g' /etc/ImageMagick-6/policy.xml
!pip install wand
!pip install pdfminer.six


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gir1.2-freedesktop
  gir1.2-gdkpixbuf-2.0 gir1.2-rsvg-2.0 gsfonts imagemagick-6-common
  libcairo-script-interpreter2 libcairo2-dev libcupsfilters1 libcupsimage2
  libdjvulibre-dev libdjvulibre-text libdjvulibre21 libgdk-pixbuf2.0-dev
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 liblcms2-dev liblqr-1-0
  liblqr-1-0-dev libmagickcore-6-arch-config libmagickcore-6-headers
  libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra libmagickcore-6.q16-dev
  libmagickwand-6-headers libmagickwand-6.q16-3 libmagickwand-6.q16-dev
  libpixman-1-dev librsvg2-dev libwmf-dev libwmf0.2-7 libxcb-shm0-dev
  poppler-data
Suggested packages:
  fonts-noto ghostscript-x libcairo2-doc inkscape libjxr-tools librsvg2-doc
  libwmf-doc libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts

In [ ]:
!wget -P /usr/share/tesseract-ocr/4.00/tessdata https://github.com/tesseract-ocr/tessdata_best/raw/master/rus.traineddata
!export TESSDATA_PREFIX=/usr/share/tesseract-ocr/4.00/tessdata

--2021-04-14 07:57:12--  https://github.com/tesseract-ocr/tessdata_best/raw/master/rus.traineddata
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/rus.traineddata [following]
--2021-04-14 07:57:12--  https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/rus.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15301764 (15M) [application/octet-stream]
Saving to: ‘/usr/share/tesseract-ocr/4.00/tessdata/rus.traineddata’

rus.traineddata     100%[===================>]  14.59M  71.2MB/s    in 0.2s    

2021-04-14 07:57:13 (71.2 MB/s) - ‘/usr/sh

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import pdfminer.high_level
tempdir = r"/tmp"
cache = os.listdir( tempdir )


from wand.resource import limits

# Use 100MB of ram before writing temp data to disk.
limits['memory'] = 1024 * 1024 * 100

for item in cache:
    if item.endswith(".jpg") or item.startswith("magick-"):
        os.remove( os.path.join( tempdir, item ) )


In [ ]:
import os
import io
from PIL import Image
import pytesseract
from wand.image import Image as wi
import gc
#pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tessdata_prefix = r"/usr/share/tesseract-ocr/4.00/tessdata"
from wand.resource import limits

# Use 100MB of ram before writing temp data to disk.
limits['memory'] = 1024 * 1024 * 100

# pdf = wi(filename = "Downloads/НК Роснефть_РСБУ_2020.pdf", resolution = 300)
# pdfImg=pdf.convert('jpeg')

imgBlobs=[]
extracted_text=[]

def Get_text_from_image(pdf_path):
    pdf=wi(filename=pdf_path,resolution=300)
    pdfImg=pdf.convert('jpeg')
    imgBlobs=[]
    extracted_text=[]
    try:
      for img in pdfImg.sequence:
          page=wi(image=img)
          imgBlobs.append(page.make_blob('jpeg'))
    finally:
        pdfImg.destroy()

    for imgBlob in imgBlobs:
        im=Image.open(io.BytesIO(imgBlob))
        text=pytesseract.image_to_string(im,lang='rus')
        extracted_text.append(text)

    return (extracted_text)

In [ ]:
#txt

## Поиск информации

In [ ]:
def find_inn(txt):
    i=0
    for txtpart in txt:
        for accrloc in find_all(txtpart.lower(), 'инн'):
            if accrloc>0:
                innm = re.findall('\d{10}', txtpart[accrloc:accrloc+50])
                if len(innm)>1:
                    inn = innm[1]
                if len(innm)>0:
                    inn = innm[0]
                print(i)
            i+=1
    return inn

In [ ]:
def find_depo(txt):
    i=0
    accr= []
    page_depo = []
    cntc = 0
    strg = []
    for txtpart in txt:
        cntc+=1
        for accrloc in find_all(txtpart.lower(), 'депозит'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+150])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_depo.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        i+=1
    return accr, page_depo, strg

In [ ]:
import re
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.lower().find(sub.lower(), start)
        if start == -1: return
        yield start
        start += len(sub)

In [ ]:
# inn = find_inn(txt)
# print(inn)
# depo = find_depo(txt)

In [ ]:
def enable_download_headless(browser,download_dir):
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd':'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': download_dir}}
    browser.execute("send_command", params)

In [ ]:
def find_rs(txt):
    i=0
    accr = []
    page_rs = []
    cntc = 0
    strg = []
    for txtpart in txt:
        cntc+=1
        for accrloc in find_all(txtpart.lower(), 'расчетные счета'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+150])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_rs.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        for accrloc in find_all(txtpart.lower(), 'текущие счета'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+150])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_rs.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        for accrloc in find_all(txtpart.lower(), 'расчетных счетах'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+150])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_rs.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        for accrloc in find_all(txtpart.lower(), 'текущих счетах'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+150])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_rs.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        for accrloc in find_all(txtpart.lower(), 'денежные эквиваленты'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+150])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_rs.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        for accrloc in find_all(txtpart.lower(), 'касс'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+150])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_rs.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
    return accr, page_rs, strg

In [ ]:
def find_gar(txt):
    i=0
    gar= []
    page_gar = []
    cntc = 0
    strg = []
    for txtpart in txt:
        cntc+=1
        for accrloc in find_all(txtpart.lower(), 'гаран'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                gar.append(''.join(accrm[0:cnt]))
                page_gar.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        i+=1
    return gar, page_gar, strg

In [ ]:
def find_por(txt):
    i=0
    accr= []
    page_accr = []
    cntc = 0
    strg = []
    for txtpart in txt:
        cntc+=1
        for accrloc in find_all(txtpart.lower(), 'обеспечение обязательств'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30]) 
        for accrloc in find_all(txtpart.lower(), 'поручительства по обязательств'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30])
        for accrloc in find_all(txtpart.lower(), '5800'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30]) 
    return accr, page_accr, strg


In [ ]:
def find_accr(txt):
    i=0
    accr= []
    page_accr = []
    cntc = 0
    strg = []
    for txtpart in txt:
        cntc+=1
        for accrloc in find_all(txtpart.lower(), 'аккре'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30]) 
        i+=1
    return accr, page_accr, strg

In [ ]:
def find_debit(txt):
    accr= []
    page_accr = []
    cntc = 0
    strg = []
    for txtpart in txt:
        cntc+=1
        for accrloc in find_all(txtpart.lower(), 'авансы выданные'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30]) 
        for accrloc in find_all(txtpart.lower(), 'прочая задолж'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)        
                strg.append(txtpart[accrloc-30:accrloc+30]) 
        for accrloc in find_all(txtpart.lower(), 'прочая дебиторская'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30]) 
        for accrloc in find_all(txtpart.lower(), 'прочие дебитор'):
            if accrloc>0:
                accrm = re.findall('\d+', txtpart[accrloc:accrloc+50])
                cnt = int(len(accrm)/3)
                accr.append(''.join(accrm[0:cnt]))
                page_accr.append(cntc)
                strg.append(txtpart[accrloc-30:accrloc+30]) 
    return accr, page_accr, strg

In [ ]:
found_inn = []
found_accr = []
pages_accr = []
strings_accr = []
found_gar = []
pages_gar = []
strings_gar = []
found_file = []
found_entries = []
found_depo = []
pages_depo = []
strings_depo = []
found_rs = []
pages_rs = []
strings_rs = []
found_debit = []
pages_debit = []
strings_debit = []
found_por = []
pages_por = []
strings_por = []
list_error_files = []
from zipfile import ZipFile
import os
from tqdm import tqdm
import pandas as pd
tempdir = r"/tmp"
cache = os.listdir( tempdir )

with os.scandir('/content/drive/MyDrive/rsbu/to_parse11/') as entries:
    for entry in tqdm(entries):
        print(entry.name)
        txt = ['']
        if entry.name[-3:]=='pdf' and entry.name not in good_files:
            try:
              txt = pdfminer.high_level.extract_text('/content/drive/MyDrive/rsbu/to_parse11/'+entry.name)
            except Exception as e:
              print('pdfminer error', e)
            if len(txt)>0 and len(txt[0]) < 100:
                try:
                    txt = Get_text_from_image('/content/drive/MyDrive/rsbu/to_parse11/'+entry.name)
                    for item in cache:
                        if item.endswith(".jpg") or item.startswith("magick-"):
                            try:
                                os.remove( os.path.join( tempdir, item ) )
                                winshell.recycle_bin().empty(confirm=False, show_progress=False, sound=False)
                            except:
                                continue
                except Exception as e:
                    list_error_files.append(entry.name)
                    print('error', e)
                    continue
            inn = entry.name[:-4]
            print(inn)
            accr, page_accr, strg_accr = find_accr(txt)
            print(page_accr)
            gar, page_gar, strg_gar = find_gar(txt)
            print(page_gar)
            depo, page_depo, strg_depo = find_depo(txt)
            print(page_depo)
            rs, page_rs, strg_rs = find_rs(txt)
            debit, page_debit, strg_debit = find_debit(txt)
            por, page_por, strg_por= find_por(txt)
            print(page_rs)
            if len(inn) == 0:
                inn = ''
            if len(accr) == 0:
                accr = ''
                page_accr = ''
                strg_accr = ''
            if len(gar) == 0:
                gar = ''
                page_gar = ''
                strg_gar = ''
            if len(depo) == 0:
                depo = ''
                page_depo = ''
                strg_depo = ''
            if len(rs) == 0:
                rs = ''
                page_rs = ''
                strg_rs = ''
            if len(debit) == 0:
                debit = ''
                page_debit = ''
                strg_debit= ''
            if len(por) == 0:
                por = ''
                page_por = ''
                strg_por = ''
            found_inn.append(inn)
            found_accr.append(accr)
            pages_accr.append(page_accr)
            strings_accr.append(strg_accr)
            found_gar.append(gar)
            pages_gar.append(page_gar)
            strings_gar.append(strg_gar)
            found_rs.append(rs)
            pages_rs.append(page_rs)
            strings_rs.append(strg_rs)
            found_depo.append(depo)
            pages_depo.append(page_depo)
            strings_depo.append(strg_depo)
            found_por.append(por)
            pages_por.append(page_por) 
            strings_por.append(strg_por) 
            found_file.append(entry.name)
            found_entries.append(txt) 
        if len(found_inn)%10 == 0 and len(found_inn)>0:
            df = pd.concat([pd.Series(found_inn), pd.Series(found_file) , pd.Series(found_accr), 
                            pd.Series(found_gar), pd.Series(found_depo), 
                            pd.Series(found_rs), pd.Series(found_debit), pd.Series(found_por),
                            pd.Series(pages_accr), pd.Series(pages_gar), pd.Series(pages_depo),
                            pd.Series(pages_rs), pd.Series(pages_debit), pd.Series(pages_por),
                            pd.Series(strings_accr), pd.Series(strings_gar), pd.Series(strings_depo),
                            pd.Series(strings_rs), pd.Series(strings_debit), pd.Series(strings_por),
              pd.Series(found_entries)], axis = 1)
            df.to_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab10.xlsx')
            pd.DataFrame(list_error_files).to_excel('/content/drive/MyDrive/rsbu/error_files10.xlsx', index = False)


0it [00:00, ?it/s]

6679049818.pdf


1it [03:57, 237.22s/it]

6679049818
[]
[]
[]
[]
6340007043.pdf


2it [14:28, 355.58s/it]

...

5918216877
[15]
[15, 25, 25]
[28]
[33, 33, 33]
5506024164.pdf


49it [7:47:09, 894.12s/it]

5506024164
[]
[11, 39]
[]
[]
5911066005.pdf


50it [8:18:36, 598.33s/it] 

5911066005
[70, 70, 70, 70, 70, 71, 72, 73, 74]
[3, 49, 52, 52]
[12, 44, 52]
[5, 52]


In [ ]:
df = pd.concat([pd.Series(found_inn), pd.Series(found_file) , pd.Series(found_accr), 
                            pd.Series(found_gar), pd.Series(found_depo), 
                            pd.Series(found_rs), pd.Series(found_debit), pd.Series(found_por),
                            pd.Series(pages_accr), pd.Series(pages_gar), pd.Series(pages_depo),
                            pd.Series(pages_rs), pd.Series(pages_debit), pd.Series(pages_por),
                            pd.Series(strings_accr), pd.Series(strings_gar), pd.Series(strings_depo),
                            pd.Series(strings_rs), pd.Series(strings_debit), pd.Series(strings_por),
              pd.Series(found_entries)], axis = 1)
df

In [ ]:
df = pd.concat([pd.Series(found_inn), pd.Series(found_file) , pd.Series(found_accr), 
                            pd.Series(found_gar), pd.Series(found_depo), 
                            pd.Series(found_rs), pd.Series(found_debit), pd.Series(found_por),
                            pd.Series(pages_accr), pd.Series(pages_gar), pd.Series(pages_depo),
                            pd.Series(pages_rs), pd.Series(pages_debit), pd.Series(pages_por),
                            pd.Series(strings_accr), pd.Series(strings_gar), pd.Series(strings_depo),
                            pd.Series(strings_rs), pd.Series(strings_debit), pd.Series(strings_por),
              pd.Series(found_entries)], axis = 1)
df.to_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab10.xlsx')
pd.DataFrame(list_error_files).to_excel('/content/drive/MyDrive/rsbu/error_files10.xlsx', index = False)

In [ ]:
import pandas as pd
df = pd.concat([pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab2.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab3.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab4.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab5.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab6.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab7.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab8.xlsx'),
                pd.read_excel('/content/drive/MyDrive/rsbu/df_accr_gar_colab9.xlsx')
                ]).drop_duplicates()
good_files = df[1]
good_files

0    7708776756.pdf
1    7706442596.pdf
2    7708185129.pdf
3    7814017341.pdf
4    8603184465.pdf
          ...      
5    7816162305.pdf
6    7826156685.pdf
7    7842349892.pdf
8    7708514824.pdf
9    7708207911.pdf
Name: 1, Length: 118, dtype: object